In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
# PermWrapper and PermLookupDict proxy the permissions system into objects that
# the template system can understand.


class PermLookupDict:
    def __init__(self, user, app_label):
        self.user, self.app_label = user, app_label

    def __repr__(self):
        return str(self.user.get_all_permissions())

    def __getitem__(self, perm_name):
        return self.user.has_perm("%s.%s" % (self.app_label, perm_name))

    def __iter__(self):
        # To fix 'item in perms.someapp' and __getitem__ interaction we need to
        # define __iter__. See #18979 for details.
        raise TypeError("PermLookupDict is not iterable.")

    def __bool__(self):
        return self.user.has_module_perms(self.app_label)


class PermWrapper:
    def __init__(self, user):
        self.user = user

    def __repr__(self):
        return f"{self.__class__.__qualname__}({self.user!r})"

    def __getitem__(self, app_label):
        return PermLookupDict(self.user, app_label)

    def __iter__(self):
        # I am large, I contain multitudes.
        raise TypeError("PermWrapper is not iterable.")

    def __contains__(self, perm_name):
        """
        Lookup by "someapp" or "someapp.someperm" in perms.
        """
        if "." not in perm_name:
            # The name refers to module.
            return bool(self[perm_name])
        app_label, perm_name = perm_name.split(".", 1)
        return self[app_label][perm_name]


In [3]:
from apps.organization.models import OrganizationUser

In [4]:
organization_user = OrganizationUser.objects.filter(organization__slug="creditium10").first()

In [5]:
organization_user.organization.name

'creditium10'

In [6]:
organization_user.user.email

'admin@admin.com'

In [7]:
organization_user_perms = PermWrapper(organization_user),

In [8]:
list(organization_user.get_all_permissions())

['inventories.change_inventory',
 'inventories.delete_inventory',
 'inventories.view_inventory',
 'inventories.add_inventory']

In [9]:
if 'inventories.change_inventory' in organization_user.get_all_permissions():
    print("can change inventories")

can change inventories


In [11]:
organization_user.is_owner

True